# Descrição do projeto prático da disciplina Aprendizado Descritivo 2025/1

- Renato Vimeiro
- 2025-06-02

## Objetivos

O principal objetivo do projeto é explorar de forma prática os conceitos vistos na disciplina. Os alunos se reunirão em grupos e simularão o processo de análise de dados através de aprendizado descritivo. Para isso, deverão obter os dados, limpá-los/pré-processá-los, e aplicar alguma(s) técnica(s) de aprendizado descritivo vista no curso. Os grupos também deverão analisar os resultados e comunicá-los de forma escrita e oral. Dessa forma, o aluno irá experienciar os desafios relacionados a análise de dados em ambientes mais reais. A apresentação escrita e oral dos resultados servirá também como experiência para o grupo tanto do ponto de vista acadêmico, na apresentação de resultados em forma de artigo e palestras públicas, quanto do ponto de vista de formulação de relatórios que possam ser absorvidos por pessoas não diretamente relacionadas ao problema investigado.

## Tarefas

São tarefas do projeto:

1. Obter dados para executar a análise. Os dados têm de ser representativos de problemas reais. Ou seja, dados obtidos de repositórios de benchmarks para Aprendizado de Máquina, tais como UCI ML Repository, não constituem dados viáveis para o projeto. Outras fontes como o Kaggle podem ser consultadas, porém, deve-se ter cuidado com o conjunto de dados escolhido. Conjuntos muito simples podem não permitir a aplicação das técnicas.
2. Limpeza dos dados. Tendo obtido os dados, os alunos deverão aplicar as transformações necessárias para tratar os dados e adequá-los à análise que será aplicada. Entre as possíveis tarefas podem estar eliminação/preenchimento de dados ausentes, mudança de formato dos dados, codificação...
3. Aplicação de aprendizado descritivo. Nessa etapa, os alunos deverão aplicar alguma(s) técnica(s) vista(s) no curso para analisar os dados e extrair padrões interessantes dos mesmos. Podem ser aplicadas tanto técnicas supervisionadas quanto não-supervisionadas. Recomenda-se que os alunos não implementem as técnicas, e tentem obter os algoritmos em algum repositório ou pacote. Existem diversos pacotes para diversos ambientes com praticamente todos os algoritmos vistos nas aulas.
4. Análise dos resultados. Após a aplicação dos métodos, os alunos devem explorar os resultados, interpretando-os em busca de padrões interessantes. Deve-se levar em consideração aqui as métricas de qualidade dos padrões.
5. Documentação/apresentação dos resultados. Os alunos devem narrar/documentar o processo que executaram durante a análise, bem como os resultados obtidos. Essa documentação se dará de forma escrita através de artigo científico, e de forma oral através de apresentação em sala de aula virtual. O artigo terá no máximo 12 páginas em formato da SBC. A apresentação terá no máximo 15min, e mais 5min para discussão com a turma.

## O que entregar

Os alunos devem criar um repositório público GitHub contendo todo o material usado nas análises (pacotes de software não devem ser incluídos no projeto, e sim referenciados para permitir a reprodução). Os dados também não precisam ser compartilhados. No entanto, caso sejam abertos, colocar o link para o local onde possam ser obtidos.

No Moodle, devem ser entregues: o artigo em pdf, os slides, e o link para o repositório com o conteúdo das análises.

## Organização

O projeto pode ser executado em grupo de até 5 integrantes. Para diminuir a carga de trabalho de todos os envolvidos, é recomendável que o projeto seja desenvolvido em grupo. Além disso, cada grupo tem que designar atividades a todos os seus membros. Devem ser evitadas divisões de tarefas interdependentes. O ideal é que todos estejam trabalhando ao mesmo tempo.

Os grupos devem eleger um representante para enviar mensagem ao professor indicando o nome completo e matrícula de todos os membros. Essa tarefa é importante para que o professor possa dimensionar o número as aulas necessárias para apresentação dos projetos. Ela deve ser realizada o mais breve possível, até o dia 15/06.

## Datas

A entrega do projeto será no dia 03/07. As apresentações serão nos dias 03, 08 e 10/07.

---

# Etapa 1: Instalar e importar bibliotecas

In [1]:
''' Instalação de bibliotecas '''

! pip install -r requirements.txt


In [10]:
''' Importando todas as bibliotecas do requirements.txt '''

import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
import pysubgroup as ps
import kagglehub as kh
import subgroups as sg
from sklearn.preprocessing import MinMaxScaler


# Etapa 2: Carregamento dos dados

In [3]:
''' Carregar os dados '''

data_path = './CC GENERAL.csv'

df = pd.read_csv(data_path)
print("Shape inicial:", df.shape)


Shape inicial: (8950, 18)


## Etapa 2.1: pre-processamento

### Etapa 2.1.1: Remover valores ausentes

In [4]:
""" Remover registros com valores nulos """

df = df.dropna()
print("Shape após remoção de nulos:", df.shape)

Shape após remoção de nulos: (8636, 18)


In [5]:
""" Normalizar colunas numéricas """

scaler = MinMaxScaler()
numeric_cols = df.columns.drop("CUST_ID")
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

### Etapa 2.1.2: Remover colunas que não serão usadas diretamente

### Etapa 2.1.3: Codificar variáveis categóricas

In [6]:
""" Criar coluna-alvo binária para segmentação (ex: ALTA UTILIZAÇÃO DE CRÉDITO) """

threshold = 0.75
df["High_Credit_Usage"] = (df["CREDIT_LIMIT"] > threshold).astype(int)

### Etapa 3: Descoberta de subgrupos com pysubgroup

In [14]:
""" Definir a tarefa """

target = ps.BinaryTarget("High_Credit_Usage", True)
# search_space = ps.create_selectors(df, ignore=["CUST_ID", "High_Credit_Usage"])
search_space = [
    selector for selector in ps.create_selectors(df, ignore=["CUST_ID", "High_Credit_Usage"])
    if len(df[selector.attribute_name].unique()) > 2
]

# Avaliador: WRAcc (Weighted Relative Accuracy)
task = ps.SubgroupDiscoveryTask(df, target, search_space, result_set_size=5, qf=ps.WRAccQF())

In [15]:
""" Executar o algoritmo SD """

result = ps.BeamSearch().execute(task)

b:\Programas\Programming\Python\Lib\site-packages\pysubgroup\binary_target.py:356: RuntimeWarning: invalid value encountered in divide
  p_subgroup = np.divide(positives_subgroup, instances_subgroup)


## Etapa 4: Visualização e análise dos subgrupos


In [ ]:
"""  """

print("\nTop 5 subgrupos encontrados:\n")

for i, sg in enumerate(result.results, 1):
    print(f"#{i}:", sg)
    print(f"   - tamanho: {sg.count()}")
    print(f"   - cobertura: {sg.coverage:.2f}")
    print(f"   - qualidade (WRAcc): {sg.quality:.4f}")
    print()


Top 5 subgrupos encontrados:

#1: (np.float64(0.0005352337694294766), (CREDIT_LIMIT>=0.2320534223706177), PositivesQF_parameters(size_sg=np.int64(1983), positives_count=np.int64(6)))
   - tamanho: 0


AttributeError: 'tuple' object has no attribute 'coverage'